# Importation et pré-processings du projet 1

## Importation

In [4]:
import pandas as pd 
import xml.etree.ElementTree as et
from lxml import etree
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import RegexpTokenizer
import re
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

tree1 = etree.parse("corpus_taln_v2.tei.xml")
#tree2 = etree.parse("exemple-article.xml")

df_cols = ["abstract_fr", "keywords_fr", "intro", "titre", "date"]

In [5]:
def text_cleaner(string):
  if string != None :
    string = string.translate(str.maketrans("\n\t", "  "))
  elif string == None:
    string = "None"
  return (string)

In [6]:
rows=[]
compteur_article = 0

root = tree1.getroot()
for article in root:
  intro = ""
  abstract_fr = ""
  keywords_fr = ""
  titre = ""

  #print(article[0].attrib)
  #print(article[0].tag)
  #print(article[1][0][0][0].text)

  abstract_fr = article[1][0][0][0].text
  keywords_fr = article[1][0][2][0].text

  abstract_fr= text_cleaner(abstract_fr)
  keywords_fr= text_cleaner(keywords_fr)

  try :
    intro = article[1][1][0][1].text
    intro = text_cleaner(intro)
  except :
    oui = 0
  
  titre = article[0][0][0][0].text
  titre = text_cleaner(titre)

  try : 
    date= article[0][0][1][2].text
    date = text_cleaner(date)
  except :
    oui = 0

  rows.append({"abstract_fr": abstract_fr, "keywords_fr":keywords_fr, "intro": intro, "titre": titre, "date":date})
  compteur_article += 1

In [7]:
df = pd.DataFrame(rows, columns = df_cols)
df

,abstract_fr,keywords_fr,intro,titre,date
0,Nous considérons dans notre travail la tâche ...,None,Le modèle de la Grammaire Applicative et Cogn...,Éléments de conception d'un système d'interpr...,1997
1,Nous donnons ici un aperçu du logiciel DECID ...,None,Dans le domaine de l'ingénierie linguistique ...,Informatisation du dictionnaire explicatif et...,1997
2,Diverses méthodes ont été proposées pour cons...,None,,Construction d'une représentation sémantique ...,1997
3,Le terme de lambda-DRT désigne un ensemble de...,None,La « Théorie des Représentations Discursives ...,Systèmes de types pour la (lambda-)DRT ascend...,1998
4,"Dans cet article, nous comparons deux modèles...",None,TAG est un formalisme initialement développé ...,Une grammaire TAG vue comme une grammaire Sen...,1998
...,...,...,...,...,...
1597,"Dans cet article, nous présentons une approch...","Curriculum d'apprentissage, transfert d'appre...",L'apprentissage humain est réalisé par étapes...,Curriculum d'apprentissage : reconnaissance d...,2019
1598,Cet article présente une méthodologie de déte...,"ellipse, anglais, corpus, sous-titres, détect...",L'ellipse renvoie à une incomplétude syntaxiq...,Détection des ellipses dans des corpus de sou...,2019
1599,La génération automatique de poésie est une t...,"génération de poésie, réseaux de neurones, fa...",La génération automatique de poésie est une t...,La génération automatique de poésie en français,2019
1600,Nous proposons une architecture neuronale ave...,"Réseaux neuronaux, modélisation de séquences,...",L'étiquetage de séquences est un problème imp...,Modèles neuronaux hybrides pour la modélisati...,2019


In [8]:
compteur_article

1602

## Génération de keywords

In [9]:
tokenizer=RegexpTokenizer("[\w]+")

replacement_patterns = [
(r'd\'', 'de '),
(r'l\'', 'le '),
(r'qu\'', 'que '),
(r',', ''),
(r'-', ' '),
(r'\.', ''),
(r'\;', '')
]

class RegexpReplacer(object):
   def __init__(self, patterns=replacement_patterns):
      self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]

   def replace(self, text):
      s = text
      for (pattern, repl) in self.patterns:
           s = re.sub(pattern, repl, s)
      return s

replacer = RegexpReplacer()

In [35]:
tokenized_abstract = []
cleaned_abstract = []
tokenized_title = []
for abstract in df['abstract_fr'] :
  tokenized_abstract.append((tokenizer.tokenize(replacer.replace(abstract.lower()))))
  cleaned_abstract.append(replacer.replace(abstract.lower()))

for title in df['titre']:
  tokenized_title.append((tokenizer.tokenize(replacer.replace(title.lower()))))

df['tokenized_abstract'] = tokenized_abstract
df['cleaned_abstract'] = cleaned_abstract
df['titre'] = tokenized_title

In [36]:
df.head()

,index,abstract_fr,keywords_fr,intro,titre,date,tokenized_abstract,cleaned_abstract,new_keywords
0,0,Nous considérons dans notre travail la tâche ...,None,Le modèle de la Grammaire Applicative et Cogn...,"[éléments, de, conception, de, un, système, de...",1997,"[nous, considérons, dans, notre, travail, la, ...",nous considérons dans notre travail la tâche ...,"[relative, grande, vitesse]"
1,1,Nous donnons ici un aperçu du logiciel DECID ...,None,Dans le domaine de l'ingénierie linguistique ...,"[informatisation, du, dictionnaire, explicatif...",1997,"[nous, donnons, ici, un, aperçu, du, logiciel,...",nous donnons ici un aperçu du logiciel decid ...,"[logiciel, développé, geta, informatiser, réda..."
2,2,Diverses méthodes ont été proposées pour cons...,None,,"[construction, de, une, représentation, sémant...",1997,"[diverses, méthodes, ont, été, proposées, pour...",diverses méthodes ont été proposées pour cons...,"[syntaxique, construction, représentation, sém..."
3,3,Le terme de lambda-DRT désigne un ensemble de...,None,La « Théorie des Représentations Discursives ...,"[systèmes, de, types, pour, la, lambda, drt, a...",1998,"[le, terme, de, lambda, drt, désigne, un, ense...",le terme de lambda drt désigne un ensemble de...,"[terme, lambda, ensemble, construire, mise, oe..."
4,4,"Dans cet article, nous comparons deux modèles...",None,TAG est un formalisme initialement développé ...,"[une, grammaire, tag, vue, comme, une, grammai...",1998,"[dans, cet, article, nous, comparons, deux, mo...",dans cet article nous comparons deux modèles ...,[tag]


In [37]:
common_word = pd.Series(' '.join(df['cleaned_abstract']).split()).value_counts()[:50]
common_word

de             16766
le              6506
des             5036
la              4953
les             3866
et              3802
un              3296
une             3175
à               3166
nous            3021
en              2582
dans            2363
pour            2303
sur             1878
du              1598
que             1490
est             1324
par             1191
corpus          1074
qui             1021
cet              967
article          955
sont             821
cette            739
ces              714
ce               704
système          700
automatique      672
analyse          661
au               643
plus             603
résultats        552
présentons       538
mots             533
notre            514
méthode          510
avec             476
approche         472
ou               460
il               449
a                441
deux             426
entre            426
français         421
partir           417
textes           412
données          386
proposons    

In [38]:
from string import digits
nltk.download('wordnet')
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('french')
lemmatizer_output=WordNetLemmatizer()
for index in range(len(df['tokenized_abstract'])) :
  df['tokenized_abstract'][index] = [lemmatizer_output.lemmatize(word.lower(), pos='v') for word in df['tokenized_abstract'][index] if word not in common_word] 
  df['tokenized_abstract'][index] = [lemmatizer_output.lemmatize(word.lower(), pos='v') for word in df['tokenized_abstract'][index] if word not in stopwords]


for index in range(len(df['titre'])) :
  df['titre'][index] = [lemmatizer_output.lemmatize(word.lower(), pos='v') for word in df['titre'][index] if word not in stopwords] 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [39]:
df.head()

,index,abstract_fr,keywords_fr,intro,titre,date,tokenized_abstract,cleaned_abstract,new_keywords
0,0,Nous considérons dans notre travail la tâche ...,None,Le modèle de la Grammaire Applicative et Cogn...,"[éléments, conception, système, interprétation...",1997,"[considérons, travail, tâche, traitement, visa...",nous considérons dans notre travail la tâche ...,"[relative, grande, vitesse]"
1,1,Nous donnons ici un aperçu du logiciel DECID ...,None,Dans le domaine de l'ingénierie linguistique ...,"[informatisation, dictionnaire, explicatif, co...",1997,"[donnons, ici, aperçu, logiciel, decid, dévelo...",nous donnons ici un aperçu du logiciel decid ...,"[logiciel, développé, geta, informatiser, réda..."
2,2,Diverses méthodes ont été proposées pour cons...,None,,"[construction, représentation, sémantique, gra...",1997,"[diverses, méthodes, proposées, construire, gr...",diverses méthodes ont été proposées pour cons...,"[syntaxique, construction, représentation, sém..."
3,3,Le terme de lambda-DRT désigne un ensemble de...,None,La « Théorie des Représentations Discursives ...,"[systèmes, type, lambda, drt, ascendante]",1998,"[terme, lambda, drt, désigne, ensemble, méthod...",le terme de lambda drt désigne un ensemble de...,"[terme, lambda, ensemble, construire, mise, oe..."
4,4,"Dans cet article, nous comparons deux modèles...",None,TAG est un formalisme initialement développé ...,"[grammaire, tag, vue, comme, grammaire, sens, ...",1998,"[comparons, modèles, linguistiques, utilisés, ...",dans cet article nous comparons deux modèles ...,[tag]


In [15]:
# Ici nous supprimons les lignes n'ayant ni abstract ni keyword

num_line = []
for index in range(0, len(df)):
  if (df['abstract_fr'][index] == 'None' or df['abstract_fr'][index] == ' ') and df['keywords_fr'][index] == 'None':
    num_line.append(index)

print(num_line)
num_line.append(1545)
df = df.drop(num_line)
df.reset_index(inplace = True)
len(df)

[17, 20, 25, 26, 27, 30, 43, 47, 49, 51, 53, 56, 58, 66, 67, 68, 70, 71, 81, 82, 83, 84, 122, 137, 193, 254, 333, 637, 686, 737, 802, 818, 833, 834, 845, 931, 932, 933, 936, 937, 938, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 952, 953, 1205, 1291, 1292]


1544

In [16]:
from collections import Counter

list_useless = ['algorithme', 'qui', 'que', 'selon', 'mais', 'texte', 'aperçu', 'textes', 'afin', 'donc', 'travail', 'tâche', 'cependant', 'alors', 'droite', 'gauche', 'sous', 'forme', 'partie', 'entre', 'partir', 'a', 'ici', 'plus', 'moins', 'exemple', 'analyse']
for i in range (0,150) :
  list_useless.append(str(i))

new_keywords = []
for index in range(0, len(df)):
  keywords_list = []
  cnt = Counter()
  for mot in df['tokenized_abstract'][index]:
    if len(mot) > 2:
      if mot not in list_useless :
        if mot[len(mot)-3] != 'o' and mot[len(mot)-2] != "n" and mot[len(mot)-1] != "s" :
          if mot[len(mot)-3] != 'a' and mot[len(mot)-2] != "i" and mot[len(mot)-1] != "t" :
            cnt[mot] += 1
  #print(cnt)
  #print(index)
  if df['abstract_fr'][index] != 'None' :
    word, max_count = cnt.most_common()[0]
    for mot in df['tokenized_abstract'][index]:
      if cnt[mot] == max_count and mot not in keywords_list:
        keywords_list.append(mot)
    if df['keywords_fr'][index] == None :
      df['keywords_fr'][index] = keywords_list
  new_keywords.append(keywords_list)

In [17]:
for index in range(0, len(df)):
  keywords_list = []
  if df['keywords_fr'][index] != 'None' :
    keywords_list = str(df['keywords_fr'][index]).split(',')
  new_keywords[index] = new_keywords[index] + keywords_list


df['new_keywords'] = new_keywords
df

,index,abstract_fr,keywords_fr,intro,titre,date,tokenized_abstract,cleaned_abstract,new_keywords
0,0,Nous considérons dans notre travail la tâche ...,None,Le modèle de la Grammaire Applicative et Cogn...,Éléments de conception d'un système d'interpr...,1997,"[considérons, travail, tâche, traitement, visa...",nous considérons dans notre travail la tâche ...,"[relative, grande, vitesse]"
1,1,Nous donnons ici un aperçu du logiciel DECID ...,None,Dans le domaine de l'ingénierie linguistique ...,Informatisation du dictionnaire explicatif et...,1997,"[donnons, ici, aperçu, logiciel, decid, dévelo...",nous donnons ici un aperçu du logiciel decid ...,"[logiciel, développé, geta, informatiser, réda..."
2,2,Diverses méthodes ont été proposées pour cons...,None,,Construction d'une représentation sémantique ...,1997,"[diverses, méthodes, proposées, construire, gr...",diverses méthodes ont été proposées pour cons...,"[syntaxique, construction, représentation, sém..."
3,3,Le terme de lambda-DRT désigne un ensemble de...,None,La « Théorie des Représentations Discursives ...,Systèmes de types pour la (lambda-)DRT ascend...,1998,"[terme, lambda, drt, désigne, ensemble, méthod...",le terme de lambda drt désigne un ensemble de...,"[terme, lambda, ensemble, construire, mise, oe..."
4,4,"Dans cet article, nous comparons deux modèles...",None,TAG est un formalisme initialement développé ...,Une grammaire TAG vue comme une grammaire Sen...,1998,"[comparons, modèles, linguistiques, utilisés, ...",dans cet article nous comparons deux modèles ...,[tag]
...,...,...,...,...,...,...,...,...,...
1539,1597,"Dans cet article, nous présentons une approch...","Curriculum d'apprentissage, transfert d'appre...",L'apprentissage humain est réalisé par étapes...,Curriculum d'apprentissage : reconnaissance d...,2019,"[bout, bout, extraction, concepts, sémantiques...",dans cet article nous présentons une approche...,"[extraction, particulier, pilotée, curriculum,..."
1540,1598,Cet article présente une méthodologie de déte...,"ellipse, anglais, corpus, sous-titres, détect...",L'ellipse renvoie à une incomplétude syntaxiq...,Détection des ellipses dans des corpus de sou...,2019,"[présente, méthodologie, détection, ellipses, ...",cet article présente une méthodologie de déte...,"[présente, détection, étiquette, morphosyntaxi..."
1541,1599,La génération automatique de poésie est une t...,"génération de poésie, réseaux de neurones, fa...",La génération automatique de poésie est une t...,La génération automatique de poésie en français,2019,"[génération, poésie, tâche, ardue, informatiqu...",la génération automatique de poésie est une t...,"[génération, génération de poésie, réseaux d..."
1542,1600,Nous proposons une architecture neuronale ave...,"Réseaux neuronaux, modélisation de séquences,...",L'étiquetage de séquences est un problème imp...,Modèles neuronaux hybrides pour la modélisati...,2019,"[architecture, neuronale, caractéristiques, pr...",nous proposons une architecture neuronale ave...,"[architecture, Réseaux neuronaux, modélisati..."


## Clustering

In [18]:
#liste de tous les keywords du dataframe
list_keywords = []
for i in range(len(df)):
  if df.new_keywords[i] != "None":
    list_keywords.append(df.new_keywords[i])

In [19]:
for i in range (1544):
  string=""
  for j in range (len(list_keywords[i])):
    string = string + str(list_keywords[i][j])
  list_keywords[i] = string

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf vectorizer de la librairies sklearn
vectorizer = TfidfVectorizer(stop_words=stopwords,max_features=1000, max_df = 0.5, use_idf = True, ngram_range=(1,3))
X = vectorizer.fit_transform(list_keywords)
print(X.shape)

terms = vectorizer.get_feature_names()

(1544, 1000)


In [21]:
from sklearn.cluster import KMeans

#clustering
num_clusters = 9
km = KMeans(n_clusters=num_clusters)
km.fit(X)
clusters = km.labels_.tolist()

In [22]:
X_dist = km.transform(X)
X_dist

array([[0.4104544 , 0.30342233, 0.38876464, ..., 0.48346403, 0.4378116 ,
        0.34011296],
       [0.4104544 , 0.30342233, 0.38876464, ..., 0.48346403, 0.4378116 ,
        0.34011296],
       [0.4104544 , 0.30342233, 0.38876464, ..., 0.48346403, 0.4378116 ,
        0.34011296],
       ...,
       [1.07064537, 1.02986863, 1.06141981, ..., 1.10029006, 1.06789043,
        1.04106714],
       [1.07494281, 1.03555331, 1.07052281, ..., 1.1028466 , 1.08636092,
        1.05269381],
       [1.08095921, 1.04174184, 1.07291097, ..., 1.10904978, 1.09018082,
        1.05570022]])

In [23]:
centroid = []
for i in range (1544):
  centroid.append(list(X_dist[i]))

df_cluster= pd.DataFrame.from_records(centroid)
df_cluster

,0,1,2,3,4,5,6,7,8
0,0.410454,0.303422,0.388765,0.417268,0.502902,0.124486,0.483464,0.437812,0.340113
1,0.410454,0.303422,0.388765,0.417268,0.502902,0.124486,0.483464,0.437812,0.340113
2,0.410454,0.303422,0.388765,0.417268,0.502902,0.124486,0.483464,0.437812,0.340113
3,0.410454,0.303422,0.388765,0.417268,0.502902,0.124486,0.483464,0.437812,0.340113
4,1.080959,1.045019,1.059204,1.083565,1.119335,1.001387,1.108149,1.091641,1.052836
...,...,...,...,...,...,...,...,...,...
1539,1.074833,1.039027,1.062911,1.042678,0.928391,0.992530,1.101802,1.087877,1.045485
1540,1.071826,1.027287,1.056932,1.062996,1.098992,0.985449,1.063282,1.065703,1.047508
1541,1.070645,1.029869,1.061420,1.072894,1.110402,0.989510,1.100290,1.067890,1.041067
1542,1.074943,1.035553,1.070523,1.078198,1.114519,0.998397,1.102847,1.086361,1.052694


# Préprocessing 


## Préprocessing sur les keywords

Tout d’abord nous stockons toutes nos dates dans y :

In [24]:
y = df['date']
y

0       1997
1       1997
2       1997
3       1998
4       1998
        ... 
1539    2019
1540    2019
1541    2019
1542    2019
1543    2019
Name: date, Length: 1544, dtype: object

Puis nous stockons une unique occurence de nos mots clefs dans une liste appelée data :

In [25]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
keywords = df['new_keywords']
data = []
for i in range (0, len(keywords)):
  for j in range (0, len(keywords[i])):
    if keywords[i][j] not in data:
      data.append(keywords[i][j])
values = np.array(data)
print(values)

['relative' 'grande' 'vitesse' ... ' expressions polylexicales vebales'
 ' niveau CECR' ' TAL pour la didactique du FLE. ']


Nous stockons dans x au format Dataframe nos keywords (ceux générés précédemment et ceux déjà existants) et nous créons pour chaque mot clef présent dans data une colonne dans x qui lui sera associée. Ces colonnes ne contiennent d’abord que des 0 

In [26]:
x = df['new_keywords']
x = x.to_frame()
column_values = [0 for i in range(0,len(x))]
for i in range(0,len(data)) :
  column_name = 'keyword' +str(i)
  x[column_name] = column_values

Puis nous remplissons ces colonnes à la manière d'un one hot encoding

In [27]:
for i in range(0,1544):
  for j in range (0, len(keywords[i])):
    for k in range (0, len(data)):
      column_name = "keyword" + str(k)
      if keywords[i][j] == data[k]:
        x[column_name][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


On supprime alors la première colonne qui contenait la liste

In [28]:
x = x.drop("new_keywords",axis = 1)
x.head()

,keyword0,keyword1,keyword2,keyword3,keyword4,keyword5,keyword6,keyword7,keyword8,keyword9,keyword10,keyword11,keyword12,keyword13,keyword14,keyword15,keyword16,keyword17,keyword18,keyword19,keyword20,keyword21,keyword22,keyword23,keyword24,keyword25,keyword26,keyword27,keyword28,keyword29,keyword30,keyword31,keyword32,keyword33,keyword34,keyword35,keyword36,keyword37,keyword38,keyword39,...,keyword5819,keyword5820,keyword5821,keyword5822,keyword5823,keyword5824,keyword5825,keyword5826,keyword5827,keyword5828,keyword5829,keyword5830,keyword5831,keyword5832,keyword5833,keyword5834,keyword5835,keyword5836,keyword5837,keyword5838,keyword5839,keyword5840,keyword5841,keyword5842,keyword5843,keyword5844,keyword5845,keyword5846,keyword5847,keyword5848,keyword5849,keyword5850,keyword5851,keyword5852,keyword5853,keyword5854,keyword5855,keyword5856,keyword5857,keyword5858
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Préprocessing sur les titres

In [40]:
titre = df['titre']
data2 = []
for i in range (0, len(titre)):
  for j in range (0, len(titre[i])):
    if titre[i][j] not in data2:
      data2.append(titre[i][j])
values2 = np.array(data2)
print(values2)

['éléments' 'conception' 'système' ... 'poésie' 'meilleur' 'polylexfle']


In [84]:
x2 = df['titre']
x2 = x2.to_frame()
column_values = [0 for i in range(0,len(x2))]
for i in range(0,len(data2)) :
  column_name = 'titre' +str(i)
  x2[column_name] = column_values

In [85]:
for i in range(0,1544):
  for j in range (0, len(titre[i])):
    for k in range (0, len(data2)):
      column_name = "titre" + str(k)
      if titre[i][j] == data2[k]:
        x2[column_name][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [86]:
x2 = x2.drop("titre",axis = 1)
x2.head()

,titre0,titre1,titre2,titre3,titre4,titre5,titre6,titre7,titre8,titre9,titre10,titre11,titre12,titre13,titre14,titre15,titre16,titre17,titre18,titre19,titre20,titre21,titre22,titre23,titre24,titre25,titre26,titre27,titre28,titre29,titre30,titre31,titre32,titre33,titre34,titre35,titre36,titre37,titre38,titre39,...,titre62,titre63,titre64,titre65,titre66,titre67,titre68,titre69,titre70,titre71,titre72,titre73,titre74,titre75,titre76,titre77,titre78,titre79,titre80,titre81,titre82,titre83,titre84,titre85,titre86,titre87,titre88,titre89,titre90,titre91,titre92,titre93,titre94,titre95,titre96,titre97,titre98,titre99,titre100,titre101
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,1,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Séparation du dataset en un train set, test set et un validation set

Enfin, nous séparons notre x et notre y en train et test avec une proportion de 80% - 20% à l'aide de train_test_split de sklearn

In [87]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

print("Dimension x_train :", x_train.shape)
print("Dimension of y_train :", y_train.shape)
print("Dimension of x_test :", x_test.shape)
print("Dimension of y_test :", y_test.shape)

Dimension x_train : (1235, 5859)
Dimension of y_train : (1235,)
Dimension of x_test : (309, 5859)
Dimension of y_test : (309,)


Puis nous divisons le x_test et le y_test en deux avec une proportion de 50% - 50% dans test et valid. Nous avons donc la répartition attendue :

In [88]:
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size = 0.50, random_state = 0)

print("Dimension of x_test :", x_test.shape)
print("Dimension of y_test :", y_test.shape)
print("Dimension of x_valid :", x_valid.shape)
print("Dimension of y_valid :", y_valid.shape)

Dimension of x_test : (154, 5859)
Dimension of y_test : (154,)
Dimension of x_valid : (155, 5859)
Dimension of y_valid : (155,)


In [89]:
y_traind, y_testd, y_validd = y_train.to_frame(), y_test.to_frame(), y_valid.to_frame()

Vérification que les dates sont relativements bien séparées de façons égales entre les 3 set

In [90]:
date_train=[]
date_valid=[]
date_test =[]

for i in range(1235):
  if y_traind.iloc[i,0] not in date_train :
    date_train.append (y_traind.iloc[i,0])
date_train.sort()
print(date_train)

for i in range(155):
  if y_validd.iloc[i,0] not in date_valid :
    date_valid.append (y_validd.iloc[i,0])
date_valid.sort()
print(date_valid)

for i in range(154):
  if y_testd.iloc[i,0] not in date_test :
    date_test.append (y_testd.iloc[i,0])
date_test.sort()
print(date_test)

['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
['1998', '1999', '2000', '2001', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2018', '2019']


Sachant que certaines dates n'apparraissent que quelques fois dans le dataset préprocessé la répartition est plutot bonne

Même procédé pour les titres et les distances aux centroid des clusters


In [91]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y, test_size = 0.20, random_state = 0)
x_test2, x_valid2, y_test2, y_valid2 = train_test_split(x_test2, y_test2, test_size = 0.50, random_state = 0)

In [92]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(df_cluster, y, test_size = 0.20, random_state = 0)
x_test3, x_valid3, y_test3, y_valid3 = train_test_split(x_test3, y_test3, test_size = 0.50, random_state = 0)

# Modèles et prédictions

## RandomForestClassifier sur les keywords

Il est important de noter que l’accuracy n’est pas une métrique représentative dans notre cas, en effet, une erreur d’un an est tout à fait acceptable. Nous regardons donc aussi la MSE comme valeur.

In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

model1 = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf= 1, max_leaf_nodes=None)
model1.fit(x_train, y_train)

y_pred1 = model1.predict(x_valid)


print("Training Accuracy :", model1.score(x_train, y_train))
print("Validating Accuracy :", model1.score(x_valid, y_valid))
print("MSE :", mean_squared_error(y_validd, y_pred1))

Training Accuracy : 0.9959514170040485
Validating Accuracy : 0.08387096774193549
MSE : 80.06451612903226


In [94]:
y_valid_list = y_validd['date'].to_numpy()
y_valid_list

array(['2015', '2015', '2006', '2009', '2004', '2005', '2013', '2006',
       '2009', '2003', '2013', '2003', '2013', '2005', '2005', '2002',
       '2011', '2014', '2013', '2001', '2006', '2010', '2003', '2002',
       '2014', '2009', '2010', '2008', '2007', '2002', '1999', '2008',
       '2003', '2002', '2007', '2016', '2015', '2015', '2007', '2011',
       '2017', '2016', '2012', '2001', '2006', '2007', '2006', '2005',
       '2015', '2009', '2013', '2010', '2017', '2009', '2003', '2010',
       '2018', '2015', '2006', '2013', '1998', '2018', '2001', '2012',
       '2012', '2008', '2013', '2015', '2007', '2006', '2009', '2014',
       '2009', '2005', '2015', '2005', '2012', '2013', '2013', '2013',
       '2016', '2017', '2006', '1997', '2012', '2003', '1999', '2000',
       '2002', '2019', '2017', '2010', '2017', '2001', '2001', '2000',
       '2012', '2004', '2014', '2007', '2009', '2008', '2005', '2007',
       '2012', '2009', '2006', '1999', '2004', '2019', '2015', '2002',
      

Nous avons créée une nouvelle métrique qui nous est propre. On obtient alors la proportion de date bien prédite à un seuil de tolérance près. Elle prend en argument le seuil de tolérance (nombre d’année en valeur absolue acceptable) ainsi que les prédictions et les valeurs réelles des dates comme ce qui suit :

In [95]:
def accuracy_adaptee(n, y_pred, y_test_list):
  cmpt = 0
  for i in range (0, len(y_pred)):
    if abs(int(y_pred[i])-int(y_test_list[i])) < (n+1) :
      cmpt +=1
  return cmpt/len(y_pred)

In [96]:
print("Taux de réponse juste à deux ans près : " + str(accuracy_adaptee(2, y_pred1, y_valid_list)))
print("Taux de réponse juste à trois ans près : " + str(accuracy_adaptee(3, y_pred1, y_valid_list)))
print("Taux de réponse juste à cinq ans près : " + str(accuracy_adaptee(5, y_pred1, y_valid_list)))
print("Taux de réponse juste à sept ans près : " + str(accuracy_adaptee(7, y_pred1, y_valid_list)))
print("Taux de réponse juste à dix ans près : " + str(accuracy_adaptee(10, y_pred1, y_valid_list)))

Taux de réponse juste à deux ans près : 0.27741935483870966
Taux de réponse juste à trois ans près : 0.34838709677419355
Taux de réponse juste à cinq ans près : 0.45161290322580644
Taux de réponse juste à sept ans près : 0.567741935483871
Taux de réponse juste à dix ans près : 0.7419354838709677


Testons différents paramètres pour voir si nous pouvons améliorer cette performance :

In [97]:
meilleur_n = 100
meilleur_m = 2
meilleure_MSE = mean_squared_error(y_validd, y_pred1)
for n in range(25, 250, 25):
  for m in range(2,10):
    model1bis = RandomForestClassifier(n_estimators=n, max_depth=None, min_samples_split=m, min_samples_leaf= 1, max_leaf_nodes=None)
    model1bis.fit(x_train, y_train)
    y_pred1bis = model1.predict(x_valid)
    MSE = mean_squared_error(y_validd, y_pred1bis)
    if MSE < meilleure_MSE :
      meilleur_n = n
      meilleure_MSE = MSE
      meilleur_m = m

print("n :", meilleur_n)
print("m :", meilleur_m)
print("MSE :", meilleure_MSE)

n : 100
m : 2
MSE : 80.06451612903226


Ce résultat n’est pas vraiment un bon résultat, et il ne change pas du résultat précédent. Nous avons donc décidé de tester un autre algorithme.

## SupportVectorClassifier sur les keywords

In [98]:
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error

model2 = SVC(C= 1.0, tol =0.0001)
model2.fit(x_train, y_train)

y_pred2 = model2.predict(x_valid)

print("Training Accuracy :", model2.score(x_train, y_train))
print("Testing Accuracy :", model2.score(x_valid, y_valid))
print("MSE :", mean_squared_error(y_validd, y_pred2))

Training Accuracy : 0.9060728744939271
Testing Accuracy : 0.04516129032258064
MSE : 33.26451612903226


Nous pouvons déjà constater que la MSE est très inférieure à ce que nous obtenions avec le RandomForest Classifier, ce qui est signe que ce modèle est nettement plus précis. 

In [99]:
print("Taux de réponse juste à deux ans près : " + str(accuracy_adaptee(2, y_pred2, y_valid_list)))
print("Taux de réponse juste à trois ans près : " + str(accuracy_adaptee(3, y_pred2, y_valid_list)))
print("Taux de réponse juste à cinq ans près : " + str(accuracy_adaptee(5, y_pred2, y_valid_list)))
print("Taux de réponse juste à sept ans près : " + str(accuracy_adaptee(7, y_pred2, y_valid_list)))
print("Taux de réponse juste à dix ans près : " + str(accuracy_adaptee(10, y_pred2, y_valid_list)))

Taux de réponse juste à deux ans près : 0.32903225806451614
Taux de réponse juste à trois ans près : 0.4258064516129032
Taux de réponse juste à cinq ans près : 0.6451612903225806
Taux de réponse juste à sept ans près : 0.7870967741935484
Taux de réponse juste à dix ans près : 0.9419354838709677


On test maintenant de changer le taux d'apprentissage sur notre SVC

In [100]:
meilleur_t = 0.001
meilleure_MSE = mean_squared_error(y_validd, y_pred2)
for t in range(1, 10):
  model2bis = SVC(C= 1.0, tol =10**(-t))
  model2bis.fit(x_train, y_train)
  y_pred2bis = model2.predict(x_valid)
  MSE = mean_squared_error(y_validd, y_pred2bis)
  if MSE < meilleure_MSE :
    meilleur_t = 10**(-t)
    meilleure_MSE = MSE

print("t :", meilleur_t)
print("MSE :", meilleure_MSE)

t : 0.001
MSE : 33.26451612903226


Testons donc avec ce seuil de tolérance :

In [101]:
model2 = SVC(C= 1.0, tol =0.001)
model2.fit(x_train, y_train)

y_pred2 = model2.predict(x_valid)

In [102]:
print("Taux de réponse juste à deux ans près : " + str(accuracy_adaptee(2, y_pred2, y_valid_list)))
print("Taux de réponse juste à trois ans près : " + str(accuracy_adaptee(3, y_pred2, y_valid_list)))
print("Taux de réponse juste à cinq ans près : " + str(accuracy_adaptee(5, y_pred2, y_valid_list)))
print("Taux de réponse juste à sept ans près : " + str(accuracy_adaptee(7, y_pred2, y_valid_list)))
print("Taux de réponse juste à dix ans près : " + str(accuracy_adaptee(10, y_pred2, y_valid_list)))

Taux de réponse juste à deux ans près : 0.32903225806451614
Taux de réponse juste à trois ans près : 0.4258064516129032
Taux de réponse juste à cinq ans près : 0.6451612903225806
Taux de réponse juste à sept ans près : 0.7870967741935484
Taux de réponse juste à dix ans près : 0.9419354838709677


## RandomForestClassifier sur la distance aux centroïdes des clusters

Appliquons maintenant les mêmes algorithmes mais aux distances de chaque article aux centroïdes de chaque cluster. En effet, celà nous permet non plus de considérer chaque keywords, mais les groupes de keywords obtenus lors de notre étape de clustering, qui seront sans doute plus pertinent d'un point de vue temporel.

In [103]:
model3 = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf= 1, max_leaf_nodes=None)
model3.fit(x_train3, y_train3)

y_pred3 = model3.predict(x_valid3)


print("Training Accuracy :", model3.score(x_train3, y_train3))
print("Validating Accuracy :", model3.score(x_valid3, y_valid3))
print("MSE :", mean_squared_error(y_validd, y_pred3))

Training Accuracy : 0.977327935222672
Validating Accuracy : 0.09032258064516129
MSE : 37.00645161290323


On constate que l'on obtient une bien meilleure MSE que pour le même algorithme appliqué aux keywords en one hot encoding.

In [104]:
print("Taux de réponse juste à deux ans près : " + str(accuracy_adaptee(2, y_pred3, y_valid_list)))
print("Taux de réponse juste à trois ans près : " + str(accuracy_adaptee(3, y_pred3, y_valid_list)))
print("Taux de réponse juste à cinq ans près : " + str(accuracy_adaptee(5, y_pred3, y_valid_list)))
print("Taux de réponse juste à sept ans près : " + str(accuracy_adaptee(7, y_pred3, y_valid_list)))
print("Taux de réponse juste à dix ans près : " + str(accuracy_adaptee(10, y_pred3, y_valid_list)))

Taux de réponse juste à deux ans près : 0.38064516129032255
Taux de réponse juste à trois ans près : 0.45161290322580644
Taux de réponse juste à cinq ans près : 0.6258064516129033
Taux de réponse juste à sept ans près : 0.7612903225806451
Taux de réponse juste à dix ans près : 0.896774193548387


Regardons maintenant si nous souhaitons améliorer nos performances :

In [105]:
meilleur_n = 100
meilleur_m = 2
meilleure_MSE = mean_squared_error(y_validd, y_pred3)
for n in range(25, 250, 25):
  for m in range(2,10):
    model3bis = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf= 1, max_leaf_nodes=None)
    model3bis.fit(x_train3, y_train3)
    y_pred3bis = model3bis.predict(x_valid3)
    MSE = mean_squared_error(y_validd, y_pred3bis)
    if MSE < meilleure_MSE :
      meilleur_n = n
      meilleure_MSE = MSE
      meilleur_m = m

print("n :", meilleur_n)
print("m :", meilleur_m)
print("MSE :", meilleure_MSE)

n : 175
m : 3
MSE : 29.83225806451613


On obtient donc un résultat plus performant, et équivalant au SVC appliqué aux keywords. Regardons ce modèle avec notre métrique :


In [106]:
model3 = RandomForestClassifier(n_estimators=175, max_depth=None, min_samples_split=5, min_samples_leaf= 1, max_leaf_nodes=None)
model3.fit(x_train3, y_train3)

y_pred3 = model3.predict(x_valid3)

In [107]:
print("Taux de réponse juste à deux ans près : " + str(accuracy_adaptee(2, y_pred3, y_valid_list)))
print("Taux de réponse juste à trois ans près : " + str(accuracy_adaptee(3, y_pred3, y_valid_list)))
print("Taux de réponse juste à cinq ans près : " + str(accuracy_adaptee(5, y_pred3, y_valid_list)))
print("Taux de réponse juste à sept ans près : " + str(accuracy_adaptee(7, y_pred3, y_valid_list)))
print("Taux de réponse juste à dix ans près : " + str(accuracy_adaptee(10, y_pred3, y_valid_list)))

Taux de réponse juste à deux ans près : 0.36774193548387096
Taux de réponse juste à trois ans près : 0.4774193548387097
Taux de réponse juste à cinq ans près : 0.6451612903225806
Taux de réponse juste à sept ans près : 0.8129032258064516
Taux de réponse juste à dix ans près : 0.9032258064516129


## SupportVectorClassifier sur la distance aux centroïdes des clusters

Regardons donc les résultats que nous obtenons avec le SVC appliqué aux distances aux centroïdes de nos clusters :

In [108]:
model4 = SVC(C= 1.0, tol =0.0001)
model4.fit(x_train3, y_train3)

y_pred4 = model4.predict(x_valid3)

print("Training Accuracy :", model4.score(x_train3, y_train3))
print("Validating Accuracy :", model4.score(x_valid3, y_valid3))
print("MSE :", mean_squared_error(y_validd, y_pred4))

Training Accuracy : 0.08987854251012145
Validating Accuracy : 0.03870967741935484
MSE : 26.270967741935483


Nous obtenons des résultats encore meilleur. Voyons si nous pouvons les améliorer.

In [109]:
print("Taux de réponse juste à deux ans près : " + str(accuracy_adaptee(2, y_pred4, y_valid_list)))
print("Taux de réponse juste à trois ans près : " + str(accuracy_adaptee(3, y_pred4, y_valid_list)))
print("Taux de réponse juste à cinq ans près : " + str(accuracy_adaptee(5, y_pred4, y_valid_list)))
print("Taux de réponse juste à sept ans près : " + str(accuracy_adaptee(7, y_pred4, y_valid_list)))
print("Taux de réponse juste à dix ans près : " + str(accuracy_adaptee(10, y_pred4, y_valid_list)))

Taux de réponse juste à deux ans près : 0.3548387096774194
Taux de réponse juste à trois ans près : 0.47096774193548385
Taux de réponse juste à cinq ans près : 0.7096774193548387
Taux de réponse juste à sept ans près : 0.832258064516129
Taux de réponse juste à dix ans près : 0.9806451612903225


In [110]:
meilleur_t = 0.0001

meilleure_MSE = mean_squared_error(y_validd, y_pred4)
for t in range(1, 10):
  model4bis = SVC(C= 1.0, tol =10**(-t))
  model4bis.fit(x_train3, y_train3)
  y_pred4bis = model4.predict(x_valid3)
  MSE = mean_squared_error(y_validd, y_pred4bis)
  if MSE < meilleure_MSE :
    meilleur_t = 10**(-t)
    meilleure_MSE = MSE

print("t :", meilleur_t)
print("MSE :", meilleure_MSE)

t : 0.0001
MSE : 26.270967741935483


Nous n'obtenons pas de meilleurs résultats en changenant le taux d'apprentissage.

## RandomForestClassifier sur les titres

Appliquons maintenant notre modèles sur les titres :

In [111]:
model5 = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf= 1, max_leaf_nodes=None)
model5.fit(x_train2, y_train2)

y_pred5 = model5.predict(x_valid2)


print("Training Accuracy :", model5.score(x_train2, y_train2))
print("Validating Accuracy :", model5.score(x_valid2, y_valid2))
print("MSE :", mean_squared_error(y_validd, y_pred5))

Training Accuracy : 0.9983805668016195
Validating Accuracy : 0.03870967741935484
MSE : 53.23225806451613


In [112]:
print("Taux de réponse juste à deux ans près : " + str(accuracy_adaptee(2, y_pred5, y_valid_list)))
print("Taux de réponse juste à trois ans près : " + str(accuracy_adaptee(3, y_pred5, y_valid_list)))
print("Taux de réponse juste à cinq ans près : " + str(accuracy_adaptee(5, y_pred5, y_valid_list)))
print("Taux de réponse juste à sept ans près : " + str(accuracy_adaptee(7, y_pred5, y_valid_list)))
print("Taux de réponse juste à dix ans près : " + str(accuracy_adaptee(10, y_pred5, y_valid_list)))

Taux de réponse juste à deux ans près : 0.23225806451612904
Taux de réponse juste à trois ans près : 0.3032258064516129
Taux de réponse juste à cinq ans près : 0.5419354838709678
Taux de réponse juste à sept ans près : 0.6967741935483871
Taux de réponse juste à dix ans près : 0.8516129032258064


In [113]:
meilleur_n = 100
meilleur_m = 2
meilleure_MSE = mean_squared_error(y_validd, y_pred5)
for n in range(25, 250, 25):
  for m in range(2,10):
    model5bis = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf= 1, max_leaf_nodes=None)
    model5bis.fit(x_train2, y_train2)
    y_pred5bis = model5bis.predict(x_valid2)
    MSE = mean_squared_error(y_validd, y_pred5bis)
    if MSE < meilleure_MSE :
      meilleur_n = n
      meilleure_MSE = MSE
      meilleur_m = m

print("n :", meilleur_n)
print("m :", meilleur_m)
print("MSE :", meilleure_MSE)

n : 175
m : 9
MSE : 47.090322580645164


In [114]:
model5 = RandomForestClassifier(n_estimators=50, max_depth=None, min_samples_split=2, min_samples_leaf= 1, max_leaf_nodes=None)
model5.fit(x_train2, y_train2)

y_pred5 = model5.predict(x_valid2)

In [115]:
print("Taux de réponse juste à deux ans près : " + str(accuracy_adaptee(2, y_pred5, y_valid_list)))
print("Taux de réponse juste à trois ans près : " + str(accuracy_adaptee(3, y_pred5, y_valid_list)))
print("Taux de réponse juste à cinq ans près : " + str(accuracy_adaptee(5, y_pred5, y_valid_list)))
print("Taux de réponse juste à sept ans près : " + str(accuracy_adaptee(7, y_pred5, y_valid_list)))
print("Taux de réponse juste à dix ans près : " + str(accuracy_adaptee(10, y_pred5, y_valid_list)))

Taux de réponse juste à deux ans près : 0.22580645161290322
Taux de réponse juste à trois ans près : 0.3032258064516129
Taux de réponse juste à cinq ans près : 0.5096774193548387
Taux de réponse juste à sept ans près : 0.6903225806451613
Taux de réponse juste à dix ans près : 0.8451612903225807


## SupportVectorClassifier sur les titres

In [116]:
model6 = SVC(C= 1.0, tol =0.0001)
model6.fit(x_train2, y_train2)

y_pred6 = model6.predict(x_valid2)

print("Training Accuracy :", model6.score(x_train2, y_train2))
print("Validating Accuracy :", model6.score(x_valid2, y_valid2))
print("MSE :", mean_squared_error(y_validd, y_pred6))

Training Accuracy : 0.2242914979757085
Validating Accuracy : 0.04516129032258064
MSE : 42.07741935483871


In [117]:
print("Taux de réponse juste à deux ans près : " + str(accuracy_adaptee(2, y_pred6, y_valid_list)))
print("Taux de réponse juste à trois ans près : " + str(accuracy_adaptee(3, y_pred6, y_valid_list)))
print("Taux de réponse juste à cinq ans près : " + str(accuracy_adaptee(5, y_pred6, y_valid_list)))
print("Taux de réponse juste à sept ans près : " + str(accuracy_adaptee(7, y_pred6, y_valid_list)))
print("Taux de réponse juste à dix ans près : " + str(accuracy_adaptee(10, y_pred6, y_valid_list)))

Taux de réponse juste à deux ans près : 0.3032258064516129
Taux de réponse juste à trois ans près : 0.3741935483870968
Taux de réponse juste à cinq ans près : 0.6064516129032258
Taux de réponse juste à sept ans près : 0.7354838709677419
Taux de réponse juste à dix ans près : 0.8838709677419355


In [118]:
meilleur_t = 0.0001

meilleure_MSE = mean_squared_error(y_validd, y_pred6)
for t in range(1, 10):
  model6bis = SVC(C= 1.0, tol =10**(-t))
  model6bis.fit(x_train2, y_train2)
  y_pred6bis = model6.predict(x_valid2)
  MSE = mean_squared_error(y_validd, y_pred6bis)
  if MSE < meilleure_MSE :
    meilleur_t = 10**(-t)
    meilleure_MSE = MSE

print("t :", meilleur_t)
print("MSE :", meilleure_MSE)

t : 0.0001
MSE : 42.07741935483871
